# Prepare Transcripts

This notebook uses the initial dataset of earnings call transcripts to clean up data errors and prepare for further processing. Earnings call transcripts were downloaded manually as described in our Initial Setup module. 

Note: the initial dataset of earnings calls is not included in this github repo due to size constraints. Anyone wanting to duplicate our results will need to perform the download manually in their own environment.

In [1]:
import numpy as np
import pandas as pd
import datetime
import re
import yfinance as yf
import os, contextlib

In [2]:
trsc = pd.read_pickle('data/motley-fool-data.pkl.bz2')
trsc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18755 entries, 0 to 18754
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        18755 non-null  object
 1   exchange    18755 non-null  object
 2   q           18755 non-null  object
 3   ticker      18755 non-null  object
 4   transcript  18755 non-null  object
dtypes: object(5)
memory usage: 732.7+ KB


In [3]:
trsc.head()

,date,exchange,q,ticker,transcript
0,"Aug 27, 2020, 9:00 p.m. ET",NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel..."
1,"Jul 30, 2020, 4:30 p.m. ET",NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...
2,"Oct 23, 2019, 5:00 p.m. ET",NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...
3,"Nov 6, 2019, 12:00 p.m. ET",NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon..."
4,"Aug 7, 2019, 8:30 a.m. ET",NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...


In [4]:
trsc.value_counts('ticker')

ticker
AAPL     62
GOOGL    58
TSLA     52
AMZN     39
DIS      38
         ..
BAND      1
GLS       1
GLUU      1
MESA      1
MRLN      1
Name: count, Length: 2876, dtype: int64

In [5]:
trsc.value_counts('q').sort_index()

q
2017-Q3       1
2017-Q4      15
2018-Q1       7
2018-Q2      16
2018-Q3      84
2018-Q4     179
2019-Q1     253
2019-Q2     383
2019-Q3     482
2019-Q4     429
2020-Q1     774
2020-Q2     894
2020-Q3     791
2020-Q4    2065
2021-Q1    2191
2021-Q2    2400
2021-Q3    2439
2021-Q4    1450
2022-Q1    1348
2022-Q2    1288
2022-Q3     646
2022-Q4     335
2023-Q1     157
2023-Q2      90
2023-Q3      38
Name: count, dtype: int64

In [6]:
# fix date column
drop_rows = []
for index, row in trsc.iterrows():
    if type(row['date']) != type(''):
        if len(row['date']) < 2:
            drop_rows.append(index)
            continue
        row['date'] = row['date'][1]

    if len(row['date']) < 15: 
        drop_rows.append(index)
        continue

    row['date'] = re.sub(r'(\d{4})[^,]', r'\1, ', row['date'])
    row['date'] = re.sub('\.', '', row['date'])
    row['date'] = row['date'].strip()
    row['date'] = re.sub(r'^(\w{3})\w+', r'\1', row['date'])

    if not re.search(r'^\w{3} \d+, \d{4}, \d+:\d{2} [ap]m \w{2}', row['date']):
        drop_rows.append(index)
        continue

    if not re.search(r'^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)', row['date']):
        drop_rows.append(index)
        continue

#print(drop_rows)

In [7]:
# drop rows with poorly formatted dates
for i in drop_rows:
    trsc.drop(i, inplace=True)

In [8]:
# fix exchange column
drop_rows = []
for index, row in trsc.iterrows():
    matchObj = re.search(r'(\w+)\s*:\s*([\w\.]+)', row['exchange'])
    if matchObj:
        exch = matchObj.group(1)
        tick = matchObj.group(2)
    else:
        print(row['exchange'])

    row['exchange'] = exch.lower()

    # drop exchanges with no data, keep NYSE/NASDAQ only
    if not re.search(r'^(NYSE|NASDAQ)$', row['exchange'], re.I):
        drop_rows.append(index)
        continue

    row['ticker'] = re.sub(r'\.', r'\-', row['ticker'])
    row['ticker'] = re.sub(r'[^\w\-]', '', row['ticker'])
    row['ticker'] = row['ticker'].lower()

In [9]:
# drop rows with non-NYSE/NASDAQ exchanges
for i in drop_rows:
    trsc.drop(i, inplace=True)

In [10]:
trsc.rename(columns={'date': 'olddate'}, inplace=True)

In [11]:
# extract AM/PM and timezone information
trsc['date'] = trsc['olddate'].str.extract(r'^(.*[pa]m)\s+')
trsc['tz'] = trsc['olddate'].str.extract(r'^.*[pa]m\s+(.*)')
trsc.drop(columns=['olddate'], inplace=True)

In [12]:
# reformat date column as a datetime
trsc['date'] = pd.to_datetime(trsc['date'], format='%b %d, %Y, %I:%M %p')

In [13]:
# identify companies holding earnings calls when markets are closed
trsc = trsc.assign(market_open=False)
for index, row in trsc.iterrows():
    h = row['date'].hour
    m = row['date'].minute
    d = row['date'].dayofweek
    if ((((h == 9) and (m >= 30)) or (h > 9)) and (h < 16)) and ((d >= 1) and (d <= 5)):
        #market_open = True
        trsc.loc[index, 'market_open'] = True

In [14]:
# only keep companies holding earnings calls when markets are closed
# daily ohlc data only supports analysis on calls happening afterhours
trsc = trsc.query('market_open == False')

In [16]:
trsc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13337 entries, 0 to 18754
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   exchange     13337 non-null  object        
 1   q            13337 non-null  object        
 2   ticker       13337 non-null  object        
 3   transcript   13337 non-null  object        
 4   date         13337 non-null  datetime64[ns]
 5   tz           13337 non-null  object        
 6   market_open  13337 non-null  bool          
dtypes: bool(1), datetime64[ns](1), object(5)
memory usage: 742.4+ KB


In [15]:
trsc.to_pickle('data/transcript_df.pkl.bz2')